# Question 1

In [3]:
# Load Libraries
%matplotlib inline

import pandas as pd

from imblearn.over_sampling import SMOTE

dat = pd.read_csv('../data/dataset_mock_midterm.csv', sep = ",")

dat = dat.drop(['date', 'severity', 'origin', 'tip_grd', 'tip_adm'], axis = 1).fillna(0) # Remove non-numerical variables and fill missing values for this notebook


In [33]:
minority_count = sum(dat['exitus'] == 1)
majority_count = sum(dat['exitus'] == 0)
print(minority_count, majority_count)

1256 31450


In [34]:
desired_ratio = 2/8
current_ratio = minority_count / majority_count
sampling_strategy = desired_ratio / current_ratio
sampling_strategy

6.259952229299363

In [61]:
sm = SMOTE(sampling_strategy =0.34,
      random_state = 0,
      k_neighbors = 5)

X_res, y_res = sm.fit_resample(dat[dat['dataset'] == 'train'].drop(['dataset', 'exitus'], axis = 1), dat[dat['dataset'] == 'train']['exitus'])


# Add dataset and exitus columns

X_res['exitus'] = y_res

X_res['dataset'] = 'train'
dat = pd.concat([X_res, dat[dat['dataset'] == 'val'], dat[dat['dataset'] == 'test']])
dat

,mortality_ratio,age,num_proc,ambulatory,expected_length,exitus,dataset
0,0.408730,12596.0,21.0,0.0,151.0,0,train
1,0.306931,20973.0,22.0,0.0,99.0,0,train
2,0.278481,19611.0,19.0,0.0,87.0,0,train
3,0.150289,13583.0,22.0,0.0,100.0,0,train
4,0.016573,18042.0,2.0,0.0,44.0,0,train
...,...,...,...,...,...,...,...
32701,0.028365,23619.0,2.0,0.0,2.0,0,test
32702,0.000606,3935.0,1.0,0.0,2.0,0,test
32703,0.040452,30163.0,4.0,0.0,2.0,0,test
32704,0.000000,29012.0,4.0,0.0,0.0,0,test


In [62]:
100*dat.groupby(['exitus'])['exitus'].agg(['count'])/dat.shape[0]

,count
exitus,
0,80.017301
1,19.982699


# Question 2

In [14]:
from sklearn.tree import DecisionTreeRegressor

# Read file
dat = pd.read_csv('../data/dataset_midterm.csv', sep = ",").fillna(0)

# 3) Define the model
model = DecisionTreeRegressor(random_state = 0)


In [5]:
# 4) Train the model

model.fit(dat[dat['dataset'] == 'train'].drop(['MEDV', 'dataset'], axis = 1), dat[dat['dataset'] == 'train'].MEDV.values)



DecisionTreeRegressor(random_state=0)

In [6]:
from sklearn.inspection import permutation_importance
import numpy as np
# perform permutation importance
importance = permutation_importance(model,

                  dat[dat['dataset'] == 'val'].drop(['MEDV', 'dataset'], axis = 1),

                  dat[dat['dataset'] == 'val'].MEDV.values,

                  random_state = 1)

importance = pd.DataFrame({'variable' :dat.drop(['MEDV', 'dataset'], axis = 1).columns.values, 'imp' : np.abs(importance.importances_mean)/np.max(np.abs(importance.importances_mean))}).sort_values(by = 'imp', ascending = False)


# Show importance scores

importance

,variable,imp
5,RM,1.000000
0,CRIM,0.527148
7,DIS,0.290793
12,LSTAT,0.235971
4,NOX,0.156554
9,TAX,0.078964
11,B,0.069776
10,PTRATIO,0.058257
2,INDUS,0.025153
6,AGE,0.001571


In [10]:
print(importance.nlargest(8, 'imp')['variable'].tolist())

['RM', 'CRIM', 'DIS', 'LSTAT', 'NOX', 'TAX', 'B', 'PTRATIO']


The 2 more important variables are RM and CRIM

In [15]:
dat = dat[['RM', 'CRIM', 'DIS', 'LSTAT', 'NOX', 'TAX', 'B', 'PTRATIO', 'MEDV', 'dataset']]

In [16]:
model.fit(dat[dat['dataset'] == 'train'].drop(['MEDV', 'dataset'], axis = 1), dat[dat['dataset'] == 'train'].MEDV.values)

DecisionTreeRegressor(random_state=0)

# Question 3